# Aprendizaje y evaluación de modelos

##  Introducción

El objetivo del ejercicio planteado consiste en comparar el rendimiento de 3 modelos de Machine Learning diferentes. Para ello, vamos a utilizar un set de datos llamado 'mtcars' sobre coches clásicos para ajustar y evaluar los 3 modelos:
1. Modelo de regresión lineal sobre todas las variables
1. Modelo de regresión lineal sobre las variables más representativas
1. Otro modelo de regresión lineal a elegir: https://scikit-learn.org/stable/modules/linear_model.html

Referencias:
- https://gallery.cortanaintelligence.com/Notebook/Evaluating-Multiple-Models-6
- https://gallery.cortanaintelligence.com/
- https://notebooks.azure.com/library/eSJDgAFMXAY

## Cargar datos
Cargar datos y mostrar dimensionalidad

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')

import pydataset
from pydataset import data
df = data('mtcars')
df.shape

Ver distribución de valores y estadística básica: media, std, max, min, percentiles...

In [ ]:
df.describe()

Mostrar valores de las variables de los 10 primeros registros del dataset

In [ ]:
df.head()

La descripción de las variables sería la siguiente:
- mpg: Miles/(US) gallon
- cyl: Number of cylinders
- disp: Displacement (cu.in.)
- hp: Gross horsepower
- drat: Rear axle ratio
- wt: Weight (1000 lbs)
- qsec: 1/4 mile time
- vs: Engine (0 = V-shaped, 1 = straight)
- am: Transmission (0 = automatic, 1 = manual)
- gear: Number of forward gears
- carb: Number of carburetors

Según el peso del coche, 'wt', y el tipo de cambio (manual/automático), 'am': ¿qué tipos de coches consumen más? Por lógica, deben sonsumir más los más pesados y con un tipo de cambio manual, pero necesitamos corroborar o justificar esa hipótesis a partir de los datos.

In [ ]:
import numpy as np

# RESPUESTA 1
print(...)

Vamos a aprender modelos de Machine Learning capaces de predecir el consumo de gasolina, 'mpg', a partir de las otras variables. Para ello, lo primero que haremos será separar el dataset en training y test sets: 
- training set para entrenar el modelo
- test set para evaluarlo

In [ ]:
from sklearn.cross_validation import train_test_split

# separar el set de características en variables predictoras, 'X', y variable objetivo, 'y'
y = df['mpg'].values
X = df.drop('mpg', 1).values
feature_names = df.drop('mpg', 1).columns

# RESPUESTA 2
# reservar 33% del dataset original (1/3) para test
X_train, X_test, y_train, y_test = ...(..., ..., ..., random_state=123)
print(X_train.shape)
print(X_test.shape)

Se puede apreciar que el número de variables predictoras es similar al número de registros disponibles que vamos a utilizar para el test. Esto nos va producir modelos sobreentrenados (overfitting) que obtienen predicciones muy buenas en la fase de training pero un rendimiento pobre en cuando a los datos de test. 

## Ajuste de los modelos

### Modelo lineal sobre todas las variables
Para ajustar el modelo lineal al conjunto de training, en primera instancia usaremos todas las variables predictoras

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
# RESPUESTA 3
lm...(..., ...)

Una vez ajustado, mostramos el valor de precisión R² respecto de los valores reales de 'mpg' vs los estimados por el modelo. También mostramos los coeficientes de ajuste para las variables predictoras.

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score

# RESPUESTA 4
# mostrar R^2 respecto del training set
r2_train = ...
print('El valor de R² para el training set es: ' + str(r2_train))

# mostrar coeficientes
param_df = pd.DataFrame({"Coefficient": [lm.intercept_] + list(lm.coef_),
                         "Feature": ['intercept'] + list(feature_names)})
param_df[['Feature', 'Coefficient']]

En base a los resultados obtenidos: ¿qué varianza estaríamos explicando con nuestro modelo respecto de los valores de la variable objetivo, 'mpg'?

In [ ]:
# RESPUESTA 5
...

Vamos ahora con el test set: ¿qué resultados obtenemos en relación al R²?

In [ ]:
# mostrar R^2 respecto del test set 
r2_test = ...
print('El valor de R² para el test set es: ' + str(r2_test))

# RESPUESTA 6
# mostrar también otras métricas de evaluación del modelo: 
# - mean absolute error (mae)
# - root mean squared error (rmse)
# - relative absolute error (rae)
# - relative squared error (rse)
mae = ...
rmse = ...
rae = ...
rse = ...

summary_df = pd.DataFrame(index = ['R-squared', 'Mean Absolute Error', 'Root Mean Squared Error',
                                   'Relative Absolute Error', 'Relative Squared Error'])
summary_df['simple linear regression, all variables'] = [r_squared, mae, rmse, rae, rse]
summary_df

En base al valor obtenido, comparándolo respecto del R² que vimos en la fase de training: ¿qué problema nos encontramos?

### Modelo lineal sobre las variables más representativas

Una posible manera de reducir el overfitting consiste en eliminar ciertas variables predictoras del dataset. Obviamente, el objetivo sería no utilizar las variables que no aportan ningún tipo de información al problema que queremos resolver. Para ello, vamos a aplicar recursive feature elimination (RFE). RFE ajusta varios modelos y compara su rendimiento, requiriendo por tanto los datasets de training y test. Reservaremos el dataset de test utilizado previamente para comparar modelos de manera equitativa. Para la evaluación del RFE necesitaremos reservar ciertos datos de training (X_train, y_train) para cada ronda de ajuste. Aplicaremos cross-validation con 'scikit-learn': https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html

También aplicaremos un escalado de las variables, para evitar que la magnitud nos estropee el modelo 

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
import seaborn as sns

# RESPUESTA 7
# escalamos cada variable a media 0 y varianza 1: x' = (x - media) / std
scaler = StandardScaler()
X_scaled = ...

# RESPUESTA 8
# eliminación recursiva de variables con CV=10, usando R² como métrica para evaluar el modelo
lm = LinearRegression()  # modelo lineal
rfe = RFECV(...) 
rfe....

# mostrar número óptimo de variables
print('Optimal number of features: {}'.format(rfe.n_features_))

# variables seleccionadas
print('Features selected: {}'.format(', '.join(np.array(feature_names)[rfe.support_].tolist())))

# obtener el orden de importancia
ranked_features, _ = zip(*sorted(zip(feature_names, rfe.ranking_.tolist()),
                                 key=lambda x: x[1],
                                 reverse=True))
print('Suggested order of feature removal: {}'.format(', '.join(ranked_features)))

In [ ]:
# plotear número de variables vs valores de R²
sns.set_style("darkgrid")
plt.figure(figsize=(8, 4))
plt.xlabel("Number of features selected")
plt.ylabel("Score")
plt.plot(range(1, len(rfe.grid_scores_) + 1), rfe.grid_scores_)
plt.show()

Los resultados deberían mostrar cómo el modelo, testeado con la estrategia CV, mejora con un número reducido pero relevante de variables. Como cabía esperar según lo visto anteriormente, el peso (wt) y el tipo de transmisión (am) estarían entre las variables a eliminar en última instancia ya que aportan cierta información al predecir nuestra variable objetivo (mpg).

Ya podemos ajustar el modelo lineal usando los datos de test que hemos utilizado en el caso anterior y así comparar de manera equitativa ambos modelos. Usaremos las mismas métricas que hemos visto antes: R², mae, rmse, rae y rse

In [ ]:
X_train_subset = X_train[:, rfe.support_]
lm2 = LinearRegression()
lm2...

X_test_part = X_test[:, rfe.support_]
predicted = ...

r_squared = ...
mae = ...
rmse = ...
rae = ...
rse = ...

summary_df['simple linear regression, selected variables'] = [r_squared, mae, rmse, rae, rse]
summary_df

¿Qué modelo obtendría mejores resultados y respecto de qué métricas? Tener en cuenta también el hecho de utilizar más o menos variables en términos de procesamiento, almacenaje, etc.

### Modelo a elegir

Seleccionar otro modelo de regresión lineal de scikit-learn y realizar el mismo análisis:
- https://scikit-learn.org/stable/modules/linear_model.html  

Aplicar nuestro modelo a todos los datos. Ajustar el modelo al dataset de entrenamiento y generar resultados para el conjunto de test

In [ ]:
import sklearn.linear_model as linear_model

my_model = linear_model...
my_model...

predicted = ...

r_squared = ...
mae = ...
rmse = ...
rae = ...
rse = ...

summary_df['my_model linear regression, all variables'] = [r_squared, mae, rmse, rae, rse]
summary_df

Comparar resultados entre sí, incluyendo los 3 modelos: ¿cuál obtiene mejores resultados? El objetivo sería que nuestro modelo sea capaz de superar los modelos de regresión lineal vistos

Seleccionar el mejor modelo, ajustarlo a todos los valores de las variables utilizadas reservando el último registro (no lo usamos para el ajuste). Guardar el modelo así entrenado disco

In [ ]:
from sklearn.externals import joblib
# RESPUESTA 9
reg_last = ...
...

Cargar el modelo generado y clasificar el registro que hemos reservado en el paso previo, comparándolo con el valor real

In [ ]:
# RESPUESTA 10
my_loaded_model = ...
my_loaded_model
print(my_loaded_model...)

Hecho! Ya tenemos nuestro modelo sencillo para futuras predicciones

## Conclusión

Con lo visto en clase y en este ejercicio, ya tenemos criterio para entrenar y generar un modelo sencillo en base a un conjunto de datos. En concreto:
- Hemos diseñado el problema a resolver, separando el set de datos en training y test para evitar overfitting
- Hemos entrenado y validado modelos, comparándolos entre sí
- Hemos probado a eliminar variables irrelevantes para el problema y ver cómo afecta a los modelos
- Hemos generado el mejor modelo a disco y lo hemos cargado para predecir un 'nuevo' registro

En la siguiente sesión, empezaremos a ver diferentes modelos según el tipo de aprendizaje a aplicar. Para empezar, modelos supervisados...